# Anki Database Reference

This is my personal reference for how to use my Anki database. It is simplified, non-exhaustive and only focuses on the data most relevant to my analyses. However, I do think that others may find this useful too, so hopefully it can help someone out there looking to dive a little deeper into the Anki database with Python and SQLite.

**Imports**

In [1]:
import sqlite3
import pandas as pd

**Magic Statements**

This allows me to use SQL in the Jupyter Notebook

In [2]:
#load ipython-sql library
%load_ext sql
#make sure the anki db is in the current directory
%sql sqlite:///collection.anki2

# Tables in the Database

Below I will only look at the `cards`, `notes`, and `revlog` tables since these are the only tables I'm currently interested in. Furthermore, in the schema sections, I will only describe some of each table's columns rather than all of them.

## `cards`

### Schema

`id` -- (***primary key***) card id in epoch miliseconds (corresponding to when the card was created)

`nid` -- (***foreign key***) the card's corresponding notes id

`type` -- 0=new, 1=learning, 2=review, 3=relearning

`reps` -- number of times the card has been reviewed

`lapses` -- the number of times the card went from a "was answered correctly" to "was answered incorrectly" state (meaning if you lapse once and then answer that same card incorrectly again, that won't cound as another lapse)

### Aside:

Because I only ever make one card per note, there are just as many cards as there are notes and each note corresponds to exactly one card and vice-versa.

In [3]:
%%sql

SELECT *
FROM cards
ORDER BY id ASC
LIMIT 5

 * sqlite:///collection.anki2
Done.


id,nid,did,ord,mod,usn,type,queue,due,ivl,factor,reps,lapses,left,odue,odid,flags,data
1591212375066,1591211441805,1597707001719,0,1614270270,555,2,-1,831,547,2500,13,1,1001,0,0,0,
1591212531185,1591212375192,1597707001719,0,1610243664,416,2,-1,611,406,2500,13,1,1001,0,0,0,
1591212849508,1591212531187,1597707001719,0,1614180069,545,2,-1,1033,750,2500,9,0,1001,0,0,0,
1591212968514,1591212849638,1597707001719,0,1616508812,612,2,-1,1135,826,2500,9,0,1001,0,0,0,
1591213182186,1591212968516,1597707001719,0,1614180129,545,2,-1,1022,739,2500,8,0,1001,0,0,0,


In [4]:
#create a dataframe
cards = %sql SELECT * FROM cards
cards = cards.DataFrame()

 * sqlite:///collection.anki2
Done.


---

## `notes`

### Schema

`id` -- (***primary key***) note id in epoch miliseconds (corresponding to when the note was created)

`tags` -- can tell whether a note is retired or not

`flds` -- data contained in the fields of each note

`sfld` -- not entirely sure, but seems to be like `fields` minus the formatting

In [5]:
%%sql

SELECT *
FROM notes
ORDER BY id ASC
LIMIT 5

 * sqlite:///collection.anki2
Done.


id,guid,mid,mod,usn,tags,flds,sfld,csum,flags,data
1591211441805,pW/b4Lrp]],1589851515791,1614270270,555,Retired,"<b><span style=""font-weight: 400;"">Où sont mes <font color=""#ef2929"">bas</font> ?</span></b><div>[sound:bas.mp3]</div> <u>(f)</u><font color=""#ef2929""> Bas</font>&nbsp;<img src=""bas.jpg"">",Où sont mes bas ?[sound:bas.mp3],1767749316,0,
1591212375192,J7+jA-3)_!,1589851515791,1610243664,416,Retired,"<b><span style=""font-weight: 400;"">J'ai les cheveux <font color=""#ef2929"">châtains</font> et un peu longs.</span></b><div>[sound:châtain.mp3]</div> <font color=""#ef2929"">Châtain</font>: Couleur brun moyen de la <font color=""#8ae234"">châtaigne</font>, souvent en parlent des cheveux.<div><br></div><div><font color=""#8ae234"">Châtaigne</font>: 'Chestnut'</div>",J'ai les cheveux châtains et un peu longs.[sound:châtain.mp3],417148383,0,
1591212531187,"Mj@;z8XJp,",1589851515791,1614180069,545,Retired,"<b><span style=""font-weight: 400;"">Il a écrit 12 livres sur le mariage, la&nbsp;<font color=""#ef2929"">douleur</font>, le courage et la beauté.&nbsp;</span></b><div>[sound:douleur.mp3]<br></div> <font color=""#ef2929"">Douleur</font>: Impression de <font color=""#8ae234"">souffrance</font>, état <font color=""#8ae234"">pénible</font> produit par un mal physique.<div><br></div><div><font color=""#8ae234"">Souffrance</font>: 'Pain'</div><div><br></div><div><font color=""#8ae234"">Pénible</font>: 'Annoying'</div>","Il a écrit 12 livres sur le mariage, la&nbsp;douleur, le courage et la beauté.&nbsp;[sound:douleur.mp3]",3487763114,0,
1591212849638,JWlp82gKx;,1589851515791,1616508812,612,Retired,"<b><span style=""font-weight: 400;"">Nous voulons vous <font color=""#ef2929"">amener</font> à l'aéroport.</span></b><div>[sound:amener.mp3]</div> <font color=""#ef2929"">Amener</font>: Conduire, transporter vers un autre endroit.",Nous voulons vous amener à l'aéroport.[sound:amener.mp3],1048561734,0,
1591212968516,"K3t,R@cMx@",1589851515791,1614180129,545,Retired,"<b><span style=""font-weight: 400;"">Mon père lui montre une <font color=""#ef2929"">liasse de billets</font>.</span></b><div>[sound:liasse.mp3]</div> <img src=""liasse de billets.jpg"">",Mon père lui montre une liasse de billets.[sound:liasse.mp3],1029174702,0,


In [6]:
#create a dataframe
notes = %sql SELECT * FROM notes
notes = notes.DataFrame()

 * sqlite:///collection.anki2
Done.


---

## `revlog`

### Schema

`id` -- (***primary key***) when I performed the review in epoch miliseconds (primary key)

`cid` -- (***foreign key***) cards id corresponding to which card was being reviewed

`ease` -- button pushed when reviewing the card
-- review:  1(wrong), 2(hard), 3(ok), 4(easy)
-- learn/relearn:   1(wrong), 2(ok), 3(easy)

`ivl` -- I believe this is the resulting interval for when one will next see the card (but then why are there negative values sometimes??)

`lastIvl` -- previous interval

`time` -- how long the review took in miliseconds up to 60000 miliseconds (60 seconds)

`type` -- what type of card at the time of review
--0 = learn, 1 = review, 2 = relearn

In [7]:
%%sql 
SELECT * 
FROM revlog
ORDER BY id DESC
LIMIT 5

 * sqlite:///collection.anki2
Done.


id,cid,usn,ease,ivl,lastIvl,factor,time,type
1629652145141,1629582268388,1132,3,1,-600,2500,4295,0
1629652140844,1629582631631,1132,3,1,-600,2500,1311,0
1629652139531,1629582441878,1132,3,1,-600,2500,1380,0
1629652138148,1629581332396,1132,3,1,-600,2500,2519,0
1629652135627,1629581804166,1132,3,1,-600,2500,950,0


In [8]:
#create a dataframe
revlog = %sql SELECT * FROM revlog
revlog = revlog.DataFrame()

 * sqlite:///collection.anki2
Done.


---

# Some useful Joins

## `cards` INNER JOIN `notes`

In [9]:
%%sql

SELECT *
FROM cards
    JOIN notes 
        ON cards.nid = notes.id
LIMIT 3

 * sqlite:///collection.anki2
Done.


id,nid,did,ord,mod,usn,type,queue,due,ivl,factor,reps,lapses,left,odue,odid,flags,data,id_1,guid,mid,mod_1,usn_1,tags,flds,sfld,csum,flags_1,data_1
1591212375066,1591211441805,1597707001719,0,1614270270,555,2,-1,831,547,2500,13,1,1001,0,0,0,,1591211441805,pW/b4Lrp]],1589851515791,1614270270,555,Retired,"<b><span style=""font-weight: 400;"">Où sont mes <font color=""#ef2929"">bas</font> ?</span></b><div>[sound:bas.mp3]</div> <u>(f)</u><font color=""#ef2929""> Bas</font>&nbsp;<img src=""bas.jpg"">",Où sont mes bas ?[sound:bas.mp3],1767749316,0,
1591212531185,1591212375192,1597707001719,0,1610243664,416,2,-1,611,406,2500,13,1,1001,0,0,0,,1591212375192,J7+jA-3)_!,1589851515791,1610243664,416,Retired,"<b><span style=""font-weight: 400;"">J'ai les cheveux <font color=""#ef2929"">châtains</font> et un peu longs.</span></b><div>[sound:châtain.mp3]</div> <font color=""#ef2929"">Châtain</font>: Couleur brun moyen de la <font color=""#8ae234"">châtaigne</font>, souvent en parlent des cheveux.<div><br></div><div><font color=""#8ae234"">Châtaigne</font>: 'Chestnut'</div>",J'ai les cheveux châtains et un peu longs.[sound:châtain.mp3],417148383,0,
1591212849508,1591212531187,1597707001719,0,1614180069,545,2,-1,1033,750,2500,9,0,1001,0,0,0,,1591212531187,"Mj@;z8XJp,",1589851515791,1614180069,545,Retired,"<b><span style=""font-weight: 400;"">Il a écrit 12 livres sur le mariage, la&nbsp;<font color=""#ef2929"">douleur</font>, le courage et la beauté.&nbsp;</span></b><div>[sound:douleur.mp3]<br></div> <font color=""#ef2929"">Douleur</font>: Impression de <font color=""#8ae234"">souffrance</font>, état <font color=""#8ae234"">pénible</font> produit par un mal physique.<div><br></div><div><font color=""#8ae234"">Souffrance</font>: 'Pain'</div><div><br></div><div><font color=""#8ae234"">Pénible</font>: 'Annoying'</div>","Il a écrit 12 livres sur le mariage, la&nbsp;douleur, le courage et la beauté.&nbsp;[sound:douleur.mp3]",3487763114,0,


### Why it could be useful
This dataset can help us analyze how many times a card was reviewed before it was retired.

---

## `revlog` LEFT JOIN `cards`

In [10]:
%%sql

SELECT *
FROM revlog
    LEFT JOIN cards 
        ON revlog.cid = cards.id
ORDER BY revlog.cid ASC
LIMIT 10
OFFSET 346

 * sqlite:///collection.anki2
Done.


id,cid,usn,ease,ivl,lastIvl,factor,time,type,id_1,nid,did,ord,mod,usn_1,type_1,queue,due,ivl_1,factor_1,reps,lapses,left,odue,odid,flags,data
1590001847693,1589928777104,0,3,3,1,2500,3518,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1589938285803,1589938259930,0,1,-60,-60,2500,18067,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1589938295563,1589938259930,0,3,-600,-60,2500,9757,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1589938307510,1589938259930,0,3,1,-600,2500,11945,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1590001868952,1589938259930,0,3,3,1,2500,21257,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1591279704089,1591212375066,0,3,-600,-60,2500,6939,0,1591212375066,1591211441805,1597707001719,0,1614270270,555,2,-1,831,547,2500,13,1,1001,0,0,0,
1591280006113,1591212375066,0,3,1,-600,2500,1317,0,1591212375066,1591211441805,1597707001719,0,1614270270,555,2,-1,831,547,2500,13,1,1001,0,0,0,
1591373106719,1591212375066,0,3,3,1,2500,1629,1,1591212375066,1591211441805,1597707001719,0,1614270270,555,2,-1,831,547,2500,13,1,1001,0,0,0,
1591625815222,1591212375066,0,3,5,3,2500,2566,1,1591212375066,1591211441805,1597707001719,0,1614270270,555,2,-1,831,547,2500,13,1,1001,0,0,0,
1592575654716,1591212375066,0,3,23,5,2500,3383,1,1591212375066,1591211441805,1597707001719,0,1614270270,555,2,-1,831,547,2500,13,1,1001,0,0,0,


### Asides:

1. The reviews (rows) for which `id_1` = None correspond to cards that have been deleted (i.e., not belonging to your main deck and thus unimportant).

2. Note that when you convert this sql query to a dataframe, every column that finished by a `_1` in the query loses this suffix and regains it's original name. You will have to rename these columns after converting to a dataframe, since otherwise you will have, for example, two `id` columns that don't refer to the same thing (the first `id` refers to the review and the second one refers to the card `id`).

### Why it could be useful

1. I could analyze the relationship between number of lapses and time spent reviewing (hypothesis being that the more often a card lapses, the longer I usually spend when reviewing that card).

2. I could also compare card.type (different from revlog.type) and time (hypothesis being that I may review some cards types faster than others)

---

## `revlog` LEFT JOIN `cards` LEFT JOIN `notes`

In [11]:
%%sql

SELECT *
FROM revlog
    LEFT JOIN cards 
        ON revlog.cid = cards.id
    LEFT JOIN notes 
        ON cards.nid = notes.id
ORDER BY revlog.cid DESC
LIMIT 3

 * sqlite:///collection.anki2
Done.


id,cid,usn,ease,ivl,lastIvl,factor,time,type,id_1,nid,did,ord,mod,usn_1,type_1,queue,due,ivl_1,factor_1,reps,lapses,left,odue,odid,flags,data,id_2,guid,mid,mod_1,usn_2,tags,flds,sfld,csum,flags_1,data_1
1629652140844,1629582631631,1132,3,1,-600,2500,1311,0,1629582631631,1629582631631,1597707001719,0,1629652140,1132,2,2,462,1,2500,2,0,1001,0,0,0,,1629582631631,O5Su085FBF,1589851515791,1629582631,1130,,"<span style=""color: rgb(255, 0, 0);"">Liminaire.&nbsp;</span><div><b><span style=""font-weight: 400;"">Nous avons désormais la preuve liminaire que le sommeil est un ingrédient fondamental de l’effort de récupération neurologique.</span></b></div><div>[sound:googletts-449a71ca-db37e278-dac676fb-a8e0eb97-e7e0eda6.mp3]<br></div> <font color=""#ff0000"">Liminaire.&nbsp;</font><a href=""https://fr.wiktionary.org/wiki/initial"">Initial</a>,&nbsp;<a href=""https://fr.wiktionary.org/wiki/pr%C3%A9alable"">préalable</a>,&nbsp;<a href=""https://fr.wiktionary.org/wiki/pr%C3%A9lude"">prélude</a>",Liminaire.&nbsp;Nous avons désormais la preuve liminaire que le sommeil est un ingrédient fondamental de l’effort de récupération neurologique.[sound:googletts-449a71ca-db37e278-dac676fb-a8e0eb97-e7e0eda6.mp3],1876403976,0,
1629652092294,1629582631631,1132,3,-600,-60,2500,28154,0,1629582631631,1629582631631,1597707001719,0,1629652140,1132,2,2,462,1,2500,2,0,1001,0,0,0,,1629582631631,O5Su085FBF,1589851515791,1629582631,1130,,"<span style=""color: rgb(255, 0, 0);"">Liminaire.&nbsp;</span><div><b><span style=""font-weight: 400;"">Nous avons désormais la preuve liminaire que le sommeil est un ingrédient fondamental de l’effort de récupération neurologique.</span></b></div><div>[sound:googletts-449a71ca-db37e278-dac676fb-a8e0eb97-e7e0eda6.mp3]<br></div> <font color=""#ff0000"">Liminaire.&nbsp;</font><a href=""https://fr.wiktionary.org/wiki/initial"">Initial</a>,&nbsp;<a href=""https://fr.wiktionary.org/wiki/pr%C3%A9alable"">préalable</a>,&nbsp;<a href=""https://fr.wiktionary.org/wiki/pr%C3%A9lude"">prélude</a>",Liminaire.&nbsp;Nous avons désormais la preuve liminaire que le sommeil est un ingrédient fondamental de l’effort de récupération neurologique.[sound:googletts-449a71ca-db37e278-dac676fb-a8e0eb97-e7e0eda6.mp3],1876403976,0,
1629652139531,1629582441878,1132,3,1,-600,2500,1380,0,1629582441878,1629582441878,1597707001719,0,1629652139,1132,2,2,462,1,2500,2,0,1001,0,0,0,,1629582441878,"MF,tOCR$S>",1589851515791,1629582441,1130,,"<span style=""color: rgb(255, 0, 0);"">Sous-tendre.&nbsp;</span><div><b><span style=""font-weight: 400;"">Cette réorganisation plastique et la genèse de nouvelles connexions sous-tendent le retour d’un certain niveau des fonctions motrices.</span></b></div><div>[sound:googletts-21c123c6-3d7b558b-182e84ba-d4a73aa6-7c2b1e34.mp3]<br></div> <font color=""#ff0000"">Sous-tendre.&nbsp;</font><a href=""https://www.linternaute.fr/dictionnaire/fr/definition/etre/""><br>Être</a>&nbsp;<a href=""https://www.linternaute.fr/dictionnaire/fr/definition/a-1/"">à</a>&nbsp;<a href=""https://www.linternaute.fr/dictionnaire/fr/definition/la-1/"">la</a>&nbsp;<a href=""https://www.linternaute.fr/dictionnaire/fr/definition/base/"">base</a>.",Sous-tendre.&nbsp;Cette réorganisation plastique et la genèse de nouvelles connexions sous-tendent le retour d’un certain niveau des fonctions motrices.[sound:googletts-21c123c6-3d7b558b-182e84ba-d4a73aa6-7c2b1e34.mp3],2851476616,0,


### Aside:

Similar to what occured above, when you convert this query to a dataframe, make sure that you rename identical column names.

### Why it could be useful

I could break the cards into two groups: those that are retired and those that are not retired but could've been retired by now. Does there exist a relationship between `lapses` and `lastIvl`? For the cards that aren't retired but could've been, is it more likely for a lapse to occur as the previous interval increases?